# CHAPTER4 データの加工と監視

## 4.16 算出プロパティで処理を含むデータを作成する

算出プロパティ`computed`は、methodsと同様データに処理を与えることができる。
`{{}}`やディレクティブにJSの式を記載することはできるが、複雑な処理を記載するとコードが見づらくなる。
```html
<div id="app">
  <p>{{ message }}</p>
  <p>{{ message.split('').reverse().join('') }}</p>
</div>
```
このような場合に算出プロパティを使うとコードの可読性を保つことができる。

### 算出プロパティの使い方
任意のデータを返り値とする関数をVueインスタンスの`computed`オプションに定義する。
```js
var app = new Vue({
  el: '#app',
  data: {
    message: 'Hello Vue.js!'
  },
  computed: {
    reversedMessage: function () {
      return this.message.split('').reverse().join('')
    }
  }
});
```
算出プロパティは、`data`オプションで定義したデータと同じように、テンプレート内で`{{}}`で参照することができる。
```html
<div id="app">
  <p>{{ message }}</p>
  <p>{{ reversedMessage }}</p>
</div>
```

### getterとsetter
Vue.jsは子テンプレートから親テンプレートのデータを直接変更することはできない。そこで、算出プロパティのgetterとsetterを使うことで、子テンプレートから親テンプレートのデータを変更することができる。

### ソート機能を利用しよう

算出プロパティ内で,`Array.prototype.sort()`を使うことで、配列をソートすることができる。
が、元の配列を変更してしまうので、算出プロパティ内でこのメソッドをそのまま使うべきではない。`Array.prototype.slice()`を使って、元の配列をコピーしてからソートすることで、元の配列を変更しないようにする。
```js
var app = new Vue({
  el: '#app',
  data: {
    list: [
      { id: 1, name: 'スライム', hp: 100 },
      { id: 2, name: 'ゴブリン', hp: 200 },
      { id: 3, name: 'ドラゴン', hp: 500 }
    ]
  },
  computed: {
    sortedList: function () {
      return this.list.slice().sort(function (a, b) {
        return a.hp - b.hp
      })
    }
  }
});
```


## 4.17 ウォッチャーでデータを監視し処理を自動化する

特定のデータや算出プロパティの状態を監視し、変化があったときに登録されたハンドラを呼び出すことができる。
算出プロパティ`computed`と異なり、 ウォッチャは何も返さない

### ウォッチャの使い方
`watch`オプションに`監視するデータ`と`ハンドラ`を定義する。比較が必要な場合は`function(新しい値, 古い値)`とすることで、新しい値と古い値を比較することができる。
```js
var app = new Vue({
    el: '#app',
    data: {
        message: 'Hello Vue.js!'
    },
    watch: {
        message: function (newVal, oldVal) {
            console.log(newVal, oldVal)
        },
        item: {
            handler: function (newVal, oldVal) {
                console.log(newVal, oldVal)
            },
            /**オブジェクトにまとめた際に以下も指定可能 */
            deep: true //ネストされたデータも監視する？
            immediate: true //初期値を監視する？
        }
    }
});
```



## 4.18 フィルタでテキストの変換処理を行う
「フィルタ」とは文字数やカンマの挿入など、テキストの変換処理を行うための機能。
`{{}}`や`v-bind`で`|（パイプ）`をつなげることで、フィルタを使うことができる。
```html
<div id="app">
    <p>{{ 対象データ | フィルタの名前 }}</p>
    or
    <p v-bind:id="対象データ | フィルタの名前"></p>
</div>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        message: 'Hello Vue.js!'
    },
    filters: {
        // フィルタの名前: function (対象データ) {
        //     return 変換後のデータ
        // }
        reverse: function (value) { //第一引数は　対象データが入る
            return value.split('').reverse().join('')
        }
    }
});
```

## 4.19 カスタムディレクティブでデータを監視しながらDOMにアクセスする

`カスタムディレクティブ`とは、`v-bind`などのディレクティブを自作できる機能のこと。
Vue.jsでは、基本的に要素やデータを変更したい場合バインディングを使うが, カスタムディレクティブを使うことでDOMのAPIなどを呼び出すことが可能になる。

### カスタムディレクティブの使い方
ディレクティブとして登録したい名前に`v-`をプレフィックスとしてつける。
```html
<div id="app">
    <p v-ディレクティブの名前></p>
</div>
```
`directives`オプションに`ディレクティブの名前`と`function`を定義する。
```js
var app = new Vue({
    el: '#app',
    data: {
        message: 'Hello Vue.js!'
    },
    directives: {
        // ディレクティブの名前: function (el, binding) {
        //     el: ディレクティブが紐づいている要素
        //     binding: ディレクティブの情報
        // }
        //各関数の定義は、定められたフックの中でできる
        ディレクティブの名前: {
            bind: function (el, binding, vnode, oldVnode) {
                // ディレクティブが初めて対象の要素に紐づいた時
            },
            inserted: function (el, binding, vnode, oldVnode) {
                // 親Nodeに挿入された時
            },
            update: function (el, binding, vnode, oldVnode) {
                // コンポーネントが更新される度。子コンポーネントが更新される前
            },
            componentUpdated: function (el, binding, vnode, oldVnode) {
                // コンポーネントが更新される度。子コンポーネントが更新された後
            },
            unbind: function (el, binding, vnode, oldVnode) {
                // ディレクティブが紐づいている要素から取り除かれた時
            }
        }
    }
});
```

## 4.20 nextTickで更新後のDOMにアクセスする

仮想DOMを使う関係上、データ更新後は非同期的にDOMが更新、レンダリングされる。

反映されたDOMにアクセスするには、`nextTick`を使う。
* nextTick：DOMの反映を待つ仕組み
```js
this.$nextTick(function () {
    // DOMが更新された後の処理
})
```
* 使用場面
    * データ更新後、描画されているDOMの高さや幅を取得する
    * データ更新後、描画されているDOMの位置を取得する
```html
<button v-on:click="clickHandler">ボタン</button>
<ul ref="list">
    <li v-for="item in items">{{ item }}</li>
</ul>
```
```js
var app = new Vue({
    el: '#app',
    data: {
        items: ['りんご', 'ばなな', 'いちご']
    },
    methods: {
        clickHandler: function () {
            /**1.表示データの更新 */
            this.items.push('メロン');
            /**2.仮想DOMのシステムを使いながら、実DOMにアクセス。更新後のDOMの高さを取得 */
            this.$nextTick(function () {
                // データ更新後、描画されているDOMの高さや幅を取得する
                console.log(this.$refs.list.clientHeight);
            });
        }
    }
});
```
* 注意点
    * 画像など外部ソースが含まれていた場合,それらのロードは`nextTick`では待機しない。
        * あくまで、Vue.jsが管理しているDOMのみを対象とする。
        * ソースのロードを待機後に実行したい場合は、`v-on:load`などのイベント内で`nextTick`を使う。
    